In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [1]:
# %load model.py
import torch
#torch.cuda.current_device()
#import torch.nn as nn
from torchvision import datasets ,transforms
#import torchvision
from matplotlib import pyplot as plt
#import numpy as np
#from google.colab.patches import cv2_imshow
#import torch.optim as optim
#from torch.autograd import Variable
#from torch.utils.tensorboard import SummaryWriter
#from tensorflow import summary
#from torchvision.utils import make_grid
#import os
#import torch
import torch.nn as nn
#from torchvision import datasets ,models,transforms
#from pathlib import Path
#from matplotlib import pyplot as plt
import numpy as np
#import torch.nn.functional as F
#from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt
from torch.nn import Linear, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d, Module
#from torch.optim import Adam
#import pandas as pd
#import os
#from os import listdir
from tqdm import tqdm_notebook as tqdm
#from PIL import Image
print(torch.__version__)

1.10.1


In [6]:
import gc

gc.collect()
torch.cuda.empty_cache()

#Learning Rate：學習率
LR = 0.0001

#Load data set
train_data = datasets.ImageFolder(
    'Trail_dataset/VGG19_train',
    transform = transforms.Compose([transforms.ToTensor()])                         
)

test_data = datasets.ImageFolder(
    'Trail_dataset/test_data',
    transform = transforms.Compose([transforms.ToTensor()])                         
)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=16,shuffle= True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1,shuffle=True)

#show labels
print(train_data.classes)
print(train_data.class_to_idx)

#check availability of gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CNN_Model(nn.Module):
    #列出需要哪些層
    def __init__(self):
        super(CNN_Model, self).__init__()
        # Convolution 1 , input_shape=(3,640,480)
        self.cnn1 = nn.Conv2d(3, 16, kernel_size=5, stride=1) 
        self.relu1 = nn.ReLU(inplace=True) 
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(0.25)
        # Convolution 2
        self.cnn2 = nn.Conv2d(16,8, kernel_size=11, stride=1) 
        self.relu2 = nn.ReLU(inplace=True) 
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(0.25)
        # Convolution 3
        self.cnn3 = nn.Conv2d(8,8, kernel_size=11, stride=1) 
        self.relu3 = nn.ReLU(inplace=True) 
        # Max pool 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        # Fully connected 1 ,#input_shape=(8*72*52)
        self.fc = nn.Linear(8 * 72 * 52, 18)     
    #列出forward的路徑，將init列出的層代入
    def forward(self, x):
        out = self.cnn1(x) 
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.dropout1(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = self.dropout2(out)
        out = self.cnn3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)
        out = out.view(out.size(0), -1) 
        out = self.fc(out) 
        return out
def train(model,n_epochs,train_loader,valid_loader,optimizer,criterion):
    train_acc_his,valid_acc_his=[],[]
    train_losses_his,valid_losses_his=[],[]
    for epoch in range(1, n_epochs+1):
        # keep track of training and validation loss
        train_loss,valid_loss = 0.0,0.0
        train_losses,valid_losses=[],[]
        train_correct,val_correct,train_total,val_total=0,0,0,0
        train_pred,train_target=torch.zeros(8,1),torch.zeros(8,1)
        val_pred,val_target=torch.zeros(8,1),torch.zeros(8,1)
        count=0
        count2=0
        print('running epoch: {}'.format(epoch))
        ###################
        # train the model #
        ###################
        model.train()
        for data, target in tqdm(train_loader):
            # move tensors to GPU if CUDA is available
            #if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            #calculate accuracy
            pred = output.data.max(dim = 1, keepdim = True)[1]
            train_correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            train_total += data.size(0)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_losses.append(loss.item()*data.size(0))
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            if count==0:
                train_pred=pred
                train_target=target.data.view_as(pred)
                count=count+1
            else:
                train_pred=torch.cat((train_pred,pred), 0)
                train_target=torch.cat((train_target,target.data.view_as(pred)), 0)
        train_pred=train_pred.cpu().view(-1).numpy().tolist()
        train_target=train_target.cpu().view(-1).numpy().tolist()
        ######################    
        # validate the model #
        ######################
        model.eval()
        for data, target in tqdm(valid_loader):
            # move tensors to GPU if CUDA is available
            #if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss =criterion(output, target)
            #calculate accuracy
            pred = output.data.max(dim = 1, keepdim = True)[1]
            val_correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            val_total += data.size(0)
            valid_losses.append(loss.item()*data.size(0))
            if count2==0:
                val_pred=pred
                val_target=target.data.view_as(pred)
                count2=count+1
            else:
                val_pred=torch.cat((val_pred,pred), 0)
                val_target=torch.cat((val_target,target.data.view_as(pred)), 0)
        val_pred=val_pred.cpu().view(-1).numpy().tolist()
        val_target=val_target.cpu().view(-1).numpy().tolist()
        
        # calculate average losses
        train_loss=np.average(train_losses)
        valid_loss=np.average(valid_losses)
        
        # calculate average accuracy
        train_acc=train_correct/train_total
        valid_acc=val_correct/val_total
        train_acc_his.append(train_acc)
        valid_acc_his.append(valid_acc)
        train_losses_his.append(train_loss)
        valid_losses_his.append(valid_loss)
# print training/validation statistics 
        print('\tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            train_loss, valid_loss))
        print('\tTraining Accuracy: {:.6f} \tValidation Accuracy: {:.6f}'.format(
            train_acc, valid_acc))
    return train_acc_his,valid_acc_his,train_losses_his,valid_losses_his,model

pretrained_weights = torch.load('C:\\Users\\sushi\\Desktop\\final_project\\test4\\weight.pth')
model1=CNN_Model()
model1 = model1.to(device)
n_epochs = 100
optimizer1 = torch.optim.Adam(model1.parameters(), lr=LR, weight_decay=0.01)
criterion = CrossEntropyLoss()
model1.load_state_dict(pretrained_weights)
train_acc_his,valid_acc_his,train_losses_his,valid_losses_his,model1=train(model1,n_epochs,train_loader,test_loader,optimizer1,criterion)
torch.save(model1.state_dict(), "retrain_weight.pth")



['L_1', 'L_2', 'L_3', 'L_4', 'L_5', 'L_6', 'L_7', 'L_8', 'R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'S_6', 'S_R']
{'L_1': 0, 'L_2': 1, 'L_3': 2, 'L_4': 3, 'L_5': 4, 'L_6': 5, 'L_7': 6, 'L_8': 7, 'R_1': 8, 'R_2': 9, 'R_3': 10, 'R_4': 11, 'R_5': 12, 'R_6': 13, 'R_7': 14, 'R_8': 15, 'S_6': 16, 'S_R': 17}
running epoch: 1


C:\Users\sushi\AppData\Local\Temp/ipykernel_3764/491830259.py:86: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data, target in tqdm(train_loader):


  0%|          | 0/358 [00:00<?, ?it/s]

C:\Users\sushi\AppData\Local\Temp/ipykernel_3764/491830259.py:119: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data, target in tqdm(valid_loader):


  0%|          | 0/852 [00:00<?, ?it/s]

...

  0%|          | 0/852 [00:00<?, ?it/s]

	Training Loss: 0.307292 	Validation Loss: 10.407426
	Training Accuracy: 0.994578 	Validation Accuracy: 0.178404
running epoch: 100


  0%|          | 0/358 [00:00<?, ?it/s]

  0%|          | 0/852 [00:00<?, ?it/s]

	Training Loss: 0.323416 	Validation Loss: 10.403187
	Training Accuracy: 0.994053 	Validation Accuracy: 0.179577
